In [ ]:
from __future__ import print_function
from ortools.sat.python import cp_model
                    
class RoomsPartialSolutionPrinter(cp_model.CpSolverSolutionCallback):
    """Print intermediate solutions."""

    def __init__(self, allocations, lectures, lecture_attendants, lecture_students, rooms_names, rooms_capacity, days, timestamps, sols):
        cp_model.CpSolverSolutionCallback.__init__(self)
        self._allocations = allocations
        self._lecture = lectures
        self._lectures = lecture_attendants
        self._students = lecture_students
        self._rooms = rooms_names
        self._capacity = rooms_capacity
        self._days = days
        self._timestamps = timestamps
        self._solutions = set(sols)
        self._solution_count = 0
        
    def on_solution_callback(self):
        if self._solution_count in self._solutions:
            print('Solution %i' % self._solution_count)
            for d in range(self._days):
                print('Day %i' % d)
                data = []
                raze = []
                for p in range(self._timestamps):
                    is_assigned = False
                    l = 0
                    while l < len(self._lectures):
                        if self.Value(self._allocations[(l, d, p)]):
                            is_assigned = True
                            print('  %s is at timestamp %i' % (self._lectures[l], p))
                            data.append(self._lectures[l])
                        l += 1   
                    if not is_assigned:
                        print('  Bad time %i to allocate' % p) 
                    for q in data:
                        raze.append(self._lecture[q]) 
                    allocations = {}
                    for i, j in enumerate(self._capacity):
                        for l, m in enumerate(raze):
                            if i == l:
                                allocations.update({j:m})
                        else:
                            pass
                    room = list(allocations.keys())
                    capacity = list(allocations.values())
    
                    stuff = []
                    cool = []
    
                    for r in room:
                        stuff.append(self._rooms[room.index(r)])
                    
                    for w in capacity:
                        res = dict((v, k) for k, v in self._lecture.items())
                        cool.append(res[w])
    
                    room_allocations = dict(zip(stuff, cool))
                    print(room_allocations)
                    data.clear()
                    raze.clear()
                    print()
            print()
        self._solution_count += 1  
        
    def solution_count(self):
            return self._solution_count
        

def main():
    # Data.
    group_a = {'AGR2101':210, 'CIV3101':215, 'CMP1101':220, 'LLD2101':225, 'YTR4101':230, 'AXR5101':235, 'GRE2103':240, 'BEE2107':245, 'POD3106':250, 'ZIP6201':255}
    group_b = {'AGR3101':115, 'CIV3103':120, 'CMP1105':125, 'LLD2106':130, 'YTR5102':135, 'AXR2109':140, 'GRE3205':145, 'BEE6108':150, 'POD7103':200, 'ZIP1106':205}
    group_c = {'VIB3304':95, 'NUR6204':100, 'ECO3201':105, 'LUG4101':110}
    group_d = {'RES2101':65, 'ENT2202':70, 'SUN1101':75, 'OOP3209':80, 'HUN4501':85, 'JIK1202':90}
    group_e = {'QUU4104':50, 'QOO4201':55, 'QII9201':60}
    
    lectures = {**group_a, **group_b, **group_c, **group_d, **group_e}
    lecture_attendants = list(lectures.keys())
    lecture_students = list(lectures.values())
    
    num_lectures = len(lectures)
    
    rooms = {'M1':100, 'M2': 200, 'M3': 300}
    rooms_names = list(rooms.keys())
    rooms_capacity = list(rooms.values())
    
    days = 7
    timestamps = 7
    
    all_days = range(days)
    all_timestamps = range(timestamps)
    all_lectures = range(num_lectures)

    # Create the model
    model = cp_model.CpModel()
    
    # Creating shift variables
    # allocations[(l, d, p)]: lecture 'l' is on day 'd' at time 'p'
    allocations = {}
    for l in all_lectures:
        for d in all_days:
            for p in all_timestamps:
                allocations[l, d, p] = model.NewBoolVar('allocations_l%i d%i p%i' %(l, d, p))
    
    # Only 3 lectures are ongoing at any given timestamp [related to number of rooms at facility]
    for d in all_days:  
        for p in all_timestamps:
            model.Add(sum(allocations[(l, d, p)] for l in all_lectures) == 3)     

    # Any lecture only happens once on a given day
    for l in all_lectures:
        for d in all_days:
            model.Add(sum(allocations[(l, d, p)] for p in all_timestamps)  <= 1)
    
    # Lectures in group c cannot be on a weekend [Saturday and Sunday]
    for d in all_days[5:7]:
        for p in all_timestamps :
            model.Add(sum(allocations[(l, d, p)] for l in all_lectures[len({**group_a, **group_b}):len({**group_a, **group_b, **group_c})]) == 0)
            
    # Lectures in group d can only be in the afternoon
    for l in all_lectures[len({**group_a, **group_b, **group_c}):len({**group_a, **group_b, **group_c, **group_d})]:
        for d in all_days:
            model.Add(sum(allocations[(l, d, p)] for p in all_timestamps[0:3]) == 0)
            
    # Lectures in group e can only be in the morning
    for l in all_lectures[len({**group_a, **group_b, **group_c, **group_d}):len({**group_a, **group_b, **group_c, **group_d, **group_e})]:
         for d in all_days:
            model.Add(sum(allocations[(l, d, p)]for p in all_timestamps[4:6]) == 0)
    
    # Lectures in group e cannot occur on Monday and Tuesday
    for d in all_days[0:3]:
        for p in all_timestamps:
            model.Add(sum(allocations[(l, d, p)] for l in all_lectures[len({**group_a, **group_b, **group_c, **group_d}):len({**group_a, **group_b, **group_c, **group_d, **group_e})]) == 0)

   # Split the constraint, one to provide days the other for timestamps but same lectures
    # for example a lecture can only be in the morning AND never on weekends

    # dealing with extra lectures. max lectures per week
    min_timestamps_per_lecture = (21 * days) // num_lectures
    max_timestamps_per_lecture = min_timestamps_per_lecture + 1
    
    for l in all_lectures:
        num_timestamps_worked = sum(allocations[(l, d, p)] for d in all_days for p in all_timestamps)
        model.Add(min_timestamps_per_lecture <= num_timestamps_worked)
        model.Add(num_timestamps_worked <= max_timestamps_per_lecture)
        
    # Creates the solver and solve.
    solver = cp_model.CpSolver()
    solver.parameters.linearization_level = 0
    
    # Display the first two solutions.
    a_few_solutions = range(2)                          
    solution_printer = RoomsPartialSolutionPrinter(allocations, lectures, lecture_attendants, lecture_students, rooms_names, rooms_capacity, days, timestamps, a_few_solutions)
    solver.SearchForAllSolutions(model, solution_printer)
   
     # Statistics.
    print()
    print('Statistics')
    print('  - conflicts       : %i' % solver.NumConflicts())
    print('  - branches        : %i' % solver.NumBranches())
    print('  - wall time       : %f s' % solver.WallTime())
    print('  - solutions found : %i' % solution_printer.solution_count())


if __name__ == '__main__':
    main()

Solution 0
Day 0
  BEE2107 is at timestamp 0
  YTR5102 is at timestamp 0
  ECO3201 is at timestamp 0
{'M1': 'BEE2107', 'M2': 'YTR5102', 'M3': 'ECO3201'}

  AXR5101 is at timestamp 1
  ZIP1106 is at timestamp 1
  LUG4101 is at timestamp 1
{'M1': 'AXR5101', 'M2': 'ZIP1106', 'M3': 'LUG4101'}

  POD3106 is at timestamp 2
  CMP1105 is at timestamp 2
  AXR2109 is at timestamp 2
{'M1': 'POD3106', 'M2': 'CMP1105', 'M3': 'AXR2109'}

  YTR4101 is at timestamp 3
  SUN1101 is at timestamp 3
  OOP3209 is at timestamp 3
{'M1': 'YTR4101', 'M2': 'SUN1101', 'M3': 'OOP3209'}

  CMP1101 is at timestamp 4
  RES2101 is at timestamp 4
  JIK1202 is at timestamp 4
{'M1': 'CMP1101', 'M2': 'RES2101', 'M3': 'JIK1202'}

  AGR2101 is at timestamp 5
  POD7103 is at timestamp 5
  ENT2202 is at timestamp 5
{'M1': 'AGR2101', 'M2': 'POD7103', 'M3': 'ENT2202'}

  GRE2103 is at timestamp 6
  VIB3304 is at timestamp 6
  NUR6204 is at timestamp 6
{'M1': 'GRE2103', 'M2': 'VIB3304', 'M3': 'NUR6204'}

Day 1
  AGR2101 is at ti